# Introduction

Trong phần này, chúng ta thử kết hợp kết quả của bài thực hành [trước](https://github.com/dangkh/face_detection/blob/master/EigenfacesRecognitionWithSVM.ipynb) với neural network. 

## Import function and library

In [7]:
from __future__ import print_function

from time import time
import logging
import matplotlib.pyplot as plt
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

%matplotlib inline

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

Using TensorFlow backend.


## Initial Data

In [8]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


## Split data

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

## Compute a PCA

In [10]:
n_components = 150

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("done in %0.3fs" % (time() - t0))


Extracting the top 150 eigenfaces from 966 faces
done in 0.286s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.023s


In [11]:
print(X_train_pca.shape)
print(y_train.shape)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
y_test = np_utils.to_categorical(y_test)

(966, 150)
(966,)
(966, 7)


In [12]:
## Create model and train

In [13]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

model = Sequential()
model.add(Dense(1024, input_dim=150, activation='relu'))
model.add(Dense(7, activation='softmax'))

epochs = 20
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              154624    
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 7175      
Total params: 161,799
Trainable params: 161,799
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
model.fit(X_train_pca, y_train, epochs=epochs, batch_size=32)

Epoch 1/20
966/966 [==============================] - 0s - loss: 1.5757 - acc: 0.4607     
Epoch 2/20
966/966 [==============================] - 0s - loss: 0.6743 - acc: 0.8592     
Epoch 3/20
966/966 [==============================] - 0s - loss: 0.3342 - acc: 0.9586     
Epoch 4/20
966/966 [==============================] - 0s - loss: 0.1902 - acc: 0.9886     
Epoch 5/20
966/966 [==============================] - 0s - loss: 0.1241 - acc: 0.9959     
Epoch 6/20
966/966 [==============================] - 0s - loss: 0.0890 - acc: 0.9990     
Epoch 7/20
966/966 [==============================] - 0s - loss: 0.0675 - acc: 1.0000     
Epoch 8/20
966/966 [==============================] - 0s - loss: 0.0544 - acc: 1.0000     
Epoch 9/20
966/966 [==============================] - 0s - loss: 0.0450 - acc: 1.0000     
Epoch 10/20
966/966 [==============================] - 0s - loss: 0.0384 - acc: 1.0000     
Epoch 11/20
966/966 [==============================] - 0s - loss: 0.0333 - acc: 1.0000   

## Test model

In [15]:
import numpy as np
print(y_test.shape)
print(X_test_pca.shape)


tmp = X_test_pca[1]
tmp = tmp.reshape(1,150)
print(np.argmax(model.predict(tmp)))
print(y_test[1])


## check 10 first test
total = 0 
for i in range(0,10):
    tmp = X_test_pca[i]
    tmp = tmp.reshape(1,150)
    res = np.argmax(model.predict(tmp))
    if y_test[i][res] == 1 :
        total += 1
print(total)

(322, 7)
(322, 150)
3
[ 0.  0.  0.  1.  0.  0.  0.]
10
